<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-03 15:04:47
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.03 L
Current:  1.32 C
-------------------
Today PnL: -62.90 K (-0.48%)
Current PnL: -16.50 L (-11.49%)
CY Booked + Current PnL: -12.02 L (-8.38%)
-------------------
Total profit:  4.23 L
Total loss:  -20.72 L
-------------------
Total Booked + Current PnL: 14.96 L (12.83%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.4%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.59 L (60.48%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.42 C (107.78%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.00,-16.23,X-LC,41.94,92820.0,17698.0,3239.0,0.45,23.56,3.49,27.87,9.0,5.46,0.71,16.41,X40,NTT,CHEMICALS
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236629.0,41937.0,7146.0,-0.86,21.54,3.02,25.21,31.0,5.87,1.81,30.85,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,58.06,101693.0,23331.0,10698.0,-2.43,29.77,10.52,43.43,70.0,2.18,0.78,36.01,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,11.33,H-SC,91.40,164280.0,26420.0,11335.0,2.05,19.16,6.90,27.39,116.0,2.33,1.26,37.45,X40,ATH,FMCG
85,UNIONBANK,123.87,163.00,-1.84,M-LC,90.32,172517.0,31677.0,12818.0,-0.94,22.49,7.43,31.59,79.0,2.47,1.32,49.86,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.00,-21.16,H-SC,13.98,156705.0,-39269.0,112091.0,3.78,-20.04,71.53,37.16,122.0,-0.35,1.20,17.03,XY24,NTT,FINANCE
72,SIS,477.00,528.00,2206.39,M-SC,81.72,62441.0,-18649.0,27318.0,2.84,-23.00,43.75,10.69,201.0,-0.68,0.48,26.55,X40N,NTT,MISC
28,GILLETTE,8109.44,10330.69,11.33,H-SC,91.40,164280.0,26420.0,11335.0,2.05,19.16,6.90,27.39,116.0,2.33,1.26,37.45,X40,ATH,FMCG
47,LALPATHLAB,2804.19,3545.00,-32.60,H-SC,43.01,155100.0,870.0,39876.0,1.95,0.56,25.71,26.42,114.0,0.02,1.19,22.05,X40N,NTT,HEALTHCARE
18,CERA,8421.60,10420.45,-17.66,X-SC,92.47,84968.0,-24513.0,50496.0,1.66,-22.39,59.43,23.73,63.0,-0.49,0.65,28.60,X40N,ATH,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,600.83,791.05,-10.56,H-MC,51.61,144538.0,8750.0,34241.0,-3.70,6.44,23.69,31.66,105.0,0.26,1.11,19.19,X40,ATH,INSURANCE
12,BATAINDIA,1550.24,2096.00,-32.17,M-SC,37.63,101924.0,-26746.0,72040.0,-2.62,-20.79,70.68,35.20,197.0,-0.37,0.78,3.99,X40,NTT,FOOTWEAR
75,SURYODAY,189.78,240.00,60.09,H-SC,66.67,113713.0,-36213.0,75892.0,-2.49,-24.15,66.74,26.46,129.0,-0.48,0.87,45.67,XR,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,58.06,101693.0,23331.0,10698.0,-2.43,29.77,10.52,43.43,70.0,2.18,0.78,36.01,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.00,35.36,M-SC,53.76,117450.0,-25029.0,44549.0,-2.23,-17.57,37.93,13.70,221.0,-0.56,0.90,38.62,X40N,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1461.46,2275.00,-13.85,X-LC,47.85,275750.0,14149.0,131478.0,-0.86,5.41,47.68,55.67,1.0,0.11,2.11,13.37,X40,BTT,IT
80,TCS,3794.03,4998.00,-18.41,X-LC,17.20,265395.0,-30539.0,124444.0,-1.37,-10.32,46.89,31.73,2.0,-0.25,2.03,5.29,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.69,X-LC,73.12,181800.0,-18510.0,77301.0,-0.44,-9.24,42.52,29.35,3.0,-0.24,1.39,25.60,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-23.00,X-LC,4.30,153224.0,-48162.0,135772.0,-0.59,-23.92,88.61,43.51,5.0,-0.35,1.17,5.68,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-16.14,X-LC,25.81,263200.0,-7147.0,41770.0,-0.91,-2.64,15.87,12.81,6.0,-0.17,2.02,8.93,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,13.28,H-SC,98.92,130974.0,33328.0,14342.0,1.26,34.13,10.95,48.82,134.0,2.32,1.00,45.50,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-1.84,M-LC,90.32,172517.0,31677.0,12818.0,-0.94,22.49,7.43,31.59,79.0,2.47,1.32,49.86,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,58.06,101693.0,23331.0,10698.0,-2.43,29.77,10.52,43.43,70.0,2.18,0.78,36.01,XY24,NTT,MISC
49,LICI,785.05,983.0,-3.65,H-LC,88.17,236629.0,41937.0,7146.0,-0.86,21.54,3.02,25.21,31.0,5.87,1.81,30.85,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3202.0,-16.23,X-LC,41.94,92820.0,17698.0,3239.0,0.45,23.56,3.49,27.87,9.0,5.46,0.71,16.41,X40,NTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,37.89,M-SC,64.52,142852.0,5274.0,89140.0,-1.37,3.83,62.40,68.62,183.0,0.06,1.09,23.84,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,13.28,H-SC,98.92,130974.0,33328.0,14342.0,1.26,34.13,10.95,48.82,134.0,2.32,1.00,45.50,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,11.33,H-SC,91.40,164280.0,26420.0,11335.0,2.05,19.16,6.90,27.39,116.0,2.33,1.26,37.45,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,5.05,H-MC,35.48,99858.0,14757.0,28350.0,-0.89,17.34,28.39,50.65,90.0,0.52,0.76,25.52,X40N,ATH,CABLES
17,CAMS,3643.00,5250.99,4.44,H-SC,70.97,117323.0,15319.0,29706.0,1.23,15.02,25.32,44.14,112.0,0.52,0.90,33.70,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,4.44,H-SC,70.97,117323.0,15319.0,29706.0,1.23,15.02,25.32,44.14,112.0,0.52,0.90,33.70,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,62.37,142227.0,13075.0,73986.0,-0.22,10.12,52.02,67.42,156.0,0.18,1.09,38.33,XR,NTT,BANKS
92,WIPRO,243.46,420.00,-11.07,M-LC,34.41,152675.0,1730.0,107727.0,-0.58,1.15,70.56,72.51,68.0,0.02,1.17,14.64,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.03,H-SC,56.99,125054.0,1718.0,135246.0,1.53,1.39,108.15,111.05,108.0,0.01,0.96,22.53,AR,ATH,MISC
1,AARTIIND,487.04,919.00,37.28,M-SC,54.84,78757.0,-2579.0,74717.0,0.20,-3.17,94.87,88.69,205.0,-0.03,0.60,32.81,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-22.76,H-SC,68.82,87645.0,-10335.0,32201.0,-0.66,-10.55,36.74,22.32,127.0,-0.32,0.67,27.12,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,58.06,101693.0,23331.0,10698.0,-2.43,29.77,10.52,43.43,70.0,2.18,0.78,36.01,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,53.76,117450.0,-25029.0,44549.0,-2.23,-17.57,37.93,13.70,221.0,-0.56,0.90,38.62,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-381.97,H-SC,100.00,119899.0,-13387.0,97981.0,0.38,-10.04,81.72,63.47,130.0,-0.14,0.92,100.98,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,13.28,H-SC,98.92,130974.0,33328.0,14342.0,1.26,34.13,10.95,48.82,134.0,2.32,1.00,45.50,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-381.97,H-SC,100.00,119899.0,-13387.0,97981.0,0.38,-10.04,81.72,63.47,130.0,-0.14,0.92,100.98,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,13.28,H-SC,98.92,130974.0,33328.0,14342.0,1.26,34.13,10.95,48.82,134.0,2.32,1.00,45.50,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6483.33,M-SC,97.85,72252.0,-21558.0,98176.0,-1.53,-22.98,135.88,81.67,214.0,-0.22,0.55,39.92,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,4.66,M-SC,96.77,171064.0,-29152.0,105187.0,-0.60,-14.56,61.49,37.97,195.0,-0.28,1.31,37.35,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-34.19,H-SC,95.70,160854.0,-78561.0,351514.0,-1.86,-32.81,218.53,114.01,113.0,-0.22,1.23,47.53,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.66
1,25,41.94
2,50,70.45


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.46
XY25    14.55
X40     14.40
AR      13.42
X40N    13.25
XR      10.35
X200     1.39
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.24
X-LC    18.65
M-SC    17.38
H-MC     8.46
M-LC     7.65
H-LC     7.61
X-MC     5.54
M-MC     5.48
X-SC     2.74
L-SC     2.26
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.73,-0.36,31.50
IT,10.11,-18.08,82.22
BANKS,9.68,-10.80,61.74
MISC,6.38,-29.99,87.49
FINANCE,5.86,-24.79,84.05
INSURANCE,5.28,3.34,25.63
PAINTS,5.26,-19.73,55.16
ELECTRICAL,5.11,-5.18,50.80
AUTO,4.29,-11.57,60.30


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2863959.0,26
AR,1229279.0,12
X40N,1217780.0,17
XR,1100376.0,14
X40,797993.0,11
XY25,475774.0,10
SR,196824.0,2
X200,77301.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2243081.0,21
M-SC,2045918.0,21
X-LC,870356.0,12
M-MC,550763.0,5
H-MC,521920.0,8
X-SC,350962.0,4
X-MC,309017.0,5
M-LC,305899.0,6
L-SC,301501.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      934982.0      6
M-SC       XY24      754875.0      6
H-SC       AR        546905.0      3
           X40N      499737.0      6
X-LC       X40       483374.0      5
M-SC       X40N      472897.0      6
M-MC       XY24      394039.0      3
M-SC       XR        293067.0      4
H-SC       XR        235780.0      4
M-SC       X40       224437.0      2
H-LC       AR        207185.0      3
M-LC       XY24      184275.0      4
H-MC       XY24      174162.0      2
X-LC       AR        172433.0      2
L-SC       XR        157872.0      2
X-MC       XY24      138567.0      2
X-LC       XY25      137248.0      4
M-SC       XY25      121379.0      1
X-SC       X40N      121293.0      2
           XR        118001.0      1
H-MC       XR        113943.0      1
X-SC       SR        111668.0      1
M-LC       XR        107727.0      1
X-MC       X40N       95503.0      2
M-SC       AR         94107.0      1
           SR         85156.0      1
M-MC       AR         82738.0      1
L-SC       XY24       81562.0      1
X-LC       X200       77301.0      1
H-MC       XY25       77116.0      1
X-MC       XY25       74947.0      1
M-MC       XR         73986.0      1
H-MC       X40        64505.0      2
           AR         63844.0      1
L-SC       AR         62067.0      1
L-LC       XY25       44041.0      1
H-MC       X40N       28350.0      1
H-SC       X40        25677.0      2
M-LC       XY25       13897.0      1
H-LC       XY25        7146.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
